matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
from theano import shared

## Stochastic / deterministic variables and model objects in PyMC3

### Model objects

In [ ]:
# We create all of our variables within model objects

with pm.Model() as model: # everything created within context of model object belongs to model now
    lambda_param = pm.Exponential("poisson_param", 1.0) 
    DGP = pm.Poisson("DGP", lambda_param)

In [ ]:
# We can look at our variables outside the model context, but they can be defined only inside it 

lambda_param

### Stochastic variables

How to define a stochastic variable: 
``python_var_name = pm.DiscreteUniform("pymc_var_name", 0, 10)``

Make the pymc_var_name descriptive because that's where the posterior distro will be later on!

In [ ]:
with pm.Model() as another_model:
    lambda_param = pm.Exponential("poisson_param", 1.0)

In [ ]:
# Each variable already has a test value
lambda_param.tag.test_value

# test_value is the starting point for sampling in the model if no other initial value is set. 
# It doesn't change the sampling. 

### Deterministic variables

In [ ]:
with pm.Model() as model:
    lambda_1 = pm.Exponential("lambda_1", 1.0)
    lambda_2 = pm.Exponential("lambda_2", 1.0)
    tau = pm.DiscreteUniform("tau", lower=0, upper=10)

new_deterministic_variable = lambda_1 + lambda_2

In [ ]:
def subtract(x, y):
    return x - y

with pm.Model() as model: 
    stochastic_1 = pm.Uniform("U_1", 0, 1)
    stochastic_2 = pm.Uniform("U_2", 0, 1)

    det_1 = pm.Deterministic("Delta", subtract(stochastic_1, stochastic_2))

### Putting it all together and looking at our distribution

In [ ]:
with pm.Model() as model: 
    lambda_param = pm.Exponential("poisson_param", 1.0) 
    DGP = pm.Poisson("DGP", lambda_param)

In [ ]:
samples = lambda_param.random(size=20000) # this is the stochastic variable we just defined above 

# plot the samples from the exponential distribution
plt.hist(samples, bins=60, normed=True, histtype="stepfilled")
plt.title("Prior distribution for $\lambda$")
plt.xlim(0, 10);

## Fitting a distribution to data

### Telling PyMC3 about our observed data

In [ ]:
my_fake_data = np.array([34, 54, 81, 23, 94]) # e.g. number of visitors to our website per day 

with pm.Model() as model:
    fixed_variable = pm.Poisson("fixed_var", 1, observed=my_fake_data)
    
print("Value of fixed variable: ", fixed_variable.tag.test_value)

### Priors far from true data, expansive priors

Let's sample some random data from a Normal distribution N(mu=0,sd=.5) and try to fit a distro to it using PyMC3. We're trying to see if PyMC3 can use the samples to guess the parameters of our distribution. 

In [ ]:
normal_sample = np.random.normal(0, 1.0, 1000) # mu = 0, sd = 1.0, num. samples = 1000

In [ ]:
# Fit a Normal distro to our data and see if we can recover our parameters 

with pm.Model() as random_normal:

    # set our priors for the parameters (stochastic nodes)
    sd = pm.HalfNormal('sd', sd=.5) # SD won't be zero so use half Normal
    mu = pm.Normal('mu', mu=2.0, sd=1.0)

    pred = pm.Normal('pred', mu=mu, sd=sd, observed=normal_sample)

In [ ]:
print("Type of mu:" , type(mu))
print("\nMean: ", mu.distribution.mean.eval())
print("\nLogp at .5: " , mu.distribution.logp(.5).eval()) # log means we can add probabilities instead of multiplying

In [ ]:
with random_normal:
    trace_normal = pm.sample(500) # MCMC, can use other algos ; sample = # of draws 

In [ ]:
pm.traceplot(trace_normal, ['mu', 'sd']);

### Priors far from true data, restrictive priors 

In [ ]:
normal_sample = np.random.normal(0, 1.0, 1000) # mu = 0, sd = 1.0, num. samples = 1000

In [ ]:
# Fit a Normal distro to our data and see if we can recover our parameters 
# This time with restrictive priors far away from the true values

with pm.Model() as random_normal:

    sd = pm.HalfNormal('sd', sd=.1) 
    mu = pm.Normal('mu', mu=2.0, sd=.1)

    pred = pm.Normal('pred', mu=mu, sd=sd, observed=normal_sample)

In [ ]:
with random_normal:
    trace_normal = pm.sample(500) # MCMC, can use other algos ; sample = # of draws 
                                   # Will use NUTS (no u-turn sampler), which says we don't have to set # steps 

In [ ]:
pm.traceplot(trace_normal, ['mu', 'sd']);

## Linear Regression 

In [ ]:
# Let's generate some fake (X,Y) data to fit a linear model

def make_random_OLS_data(num_obs, intercept, beta): 
    X = np.random.normal(1.0, 0.1, num_obs)
    epsilon = np.random.normal(0.0, 0.1, num_obs) # random noise 
    
    # Define our DGP
    y = intercept + (beta*X) + epsilon
    return X, y

In [ ]:
X_train, y_train = make_random_OLS_data(num_obs=1000,intercept=1.5,beta=.25)

In [ ]:
plt.scatter(X_train, y_train, alpha=.1)

In [ ]:
with pm.Model() as OLS:

    # Set the priors on the parameters of OLS 
    intercept = pm.Normal('intercept', mu=0.75, sd=1.) # actual = 1.5
    beta = pm.Normal('beta', mu=0.0, sd=1.) # actual = .25
    mu = (beta*X_train) + intercept
    sd = pm.HalfNormal('sd', sd=.1)
    
    y = pm.Normal('y_est', mu=mu, sd=sd, observed=y_train)

In [ ]:
with OLS:
    trace = pm.sample(2500)

In [ ]:
pm.traceplot(trace, ['intercept', 'beta', 'sd']); 
# intercept: true = 1.5 | prior = .75
# beta: true = .25 | prior = 0.
# true sd = .1

**ppc = posterior predictive check**

In [ ]:
# ppc = posterior predictive check 
# Gelman: "simulating replicated data under the fitted model and then comparing these to the observed data"
# Use posterior predictive checks to "look for systematic discrepancies between real and simulated data"

with OLS:
    
    # sample the posterior to fit
    ppc = pm.sample_ppc(trace, samples=2000) 

print("\nCreate 2000 samples per each of our 1000 observation:", ppc['y_est'].shape)

In [ ]:
# Use the mean of the 2000 samples per observation to fit our OLS  
y_hat = ppc['y_est']
y_hat_mean = np.reshape(np.mean(y_hat, axis=0), [-1])

plt.scatter(X_train, y_train)
plt.scatter(X_train, y_hat_mean)
plt.title("Fitting an OLS via Bayesian Inference");

## Hierarchical Linear Model

In [ ]:
# generate random data 

NUM_GROUPS = 10; 

def generate_hierarchical_data(num_obs_per_group, num_groups, pop_intercept, pop_beta, pop_sd):
    
    group_lvl_params = []
    # numpy arrays most efficient when you create empty ones of the size you'll need at the start
    # since they're assigned contiguous memory 
    Y = np.zeros(shape=(num_groups*num_obs_per_group,1))
    X = np.zeros(shape=(num_groups*num_obs_per_group,1))
    group_id = np.zeros(shape=(num_groups*num_obs_per_group,1))

    for _ in range(num_groups):
        intercept_grp = np.random.normal(pop_intercept, pop_sd)
        beta_grp = np.random.normal(pop_beta, pop_sd)
        group_lvl_params.append([intercept_grp, beta_grp])
        
    print("Group level parameters: \n" , group_lvl_params)
        
    for i, group in enumerate(group_lvl_params):
        indiv_epsilon = np.reshape(np.random.normal(0.0, 0.01, num_obs_per_group), [-1, 1])
        y = group[1]*indiv_epsilon + group[0] # use group parameters and noise (epsilon) to create individual obs

        X[i*num_obs_per_group:i*num_obs_per_group+num_obs_per_group,0] = indiv_epsilon.ravel()
        group_id[i*num_obs_per_group:i*num_obs_per_group+num_obs_per_group,0] = np.ones_like(indiv_epsilon).ravel() * i
        Y[i*num_obs_per_group:i*num_obs_per_group+num_obs_per_group,0] = y.ravel()
    
    return X, Y, group_id

def RMSE(Y,y_hat): 
    return np.sqrt(np.mean((y_hat - Y) ** 2.0))

X, Y, group_id = generate_hierarchical_data(num_obs_per_group=50,num_groups=NUM_GROUPS,pop_intercept=2.3,
                                  pop_beta=4.5,pop_sd=.1)

In [ ]:
X_shared = shared(X) # from Theano, keeps data in memory 
Y_shared = shared(Y)

### Unpooled

In [ ]:
with pm.Model() as FE_unpooled:

    intercept = pm.Normal('intercept', mu=1, sd=.5, shape=NUM_GROUPS)
    beta = pm.Normal('beta', mu=2, sd=.5, shape=NUM_GROUPS)
    mu = (beta[group_id.astype(int)] * X_shared) + intercept[group_id.astype(int)]
    
    y = pm.Normal('y', mu, observed=Y)

In [ ]:
with FE_unpooled:
    unpooled_trace = pm.sample(2000, tune=500) # tune will use steps but discrad them 

In [ ]:
pm.traceplot(unpooled_trace, ['intercept', 'beta']);

In [ ]:
#### RMSE of unpooled 
unpooled_sample = pm.sample_ppc(unpooled_trace, model=FE_unpooled, size=1000)
y_hat_unpooled = np.mean(unpooled_sample['y'], axis=1)

In [ ]:
unpooled_RMSE = RMSE(Y,y_hat_unpooled)

print("RMSE of unpooled: ", unpooled_RMSE)

### Pooled

In [ ]:
with pm.Model() as RE_pooled:

    pop_intercept = pm.Normal('group_intercept', 1, sd=5., shape=1)
    pop_intercept_sigma = pm.HalfNormal('group_intercept_sigma', sd=.5)
    pop_beta = pm.Normal('group_beta', 4, sd=5., shape=1)
    pop_beta_sigma = pm.HalfNormal('group_beta_sigma', sd=.5)
    
    intercept = pm.Normal('intercept', mu=pop_intercept, sd=pop_intercept_sigma, shape=NUM_GROUPS)
    beta = pm.Normal('beta', mu=pop_beta, sd=pop_beta_sigma, shape=NUM_GROUPS)
    mu = (beta[group_id.astype(int)] * X_shared) + intercept[group_id.astype(int)]
    
    y = pm.Normal('y', mu, observed=Y_shared)

In [ ]:
with RE_pooled:
    pooled_trace = pm.sample(2000, tune=2000)

In [ ]:
pm.traceplot(pooled_trace, ['group_intercept', 'group_intercept_sigma',
                     'group_beta', 'group_beta_sigma', 'intercept', 'beta']);

In [ ]:
#### RMSE of pooled 
pooled_sample = pm.sample_ppc(pooled_trace, model=RE_pooled, size=1000)
y_hat_pooled = np.mean(pooled_sample['y'], axis=1)

pooled_RMSE = RMSE(Y,y_hat_pooled)

print("RMSE of unpooled: ", unpooled_RMSE)
print("RMSE of pooled: ", pooled_RMSE)